In [7]:
import numpy as np
from sklearn.impute import SimpleImputer
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from scipy import optimize
from torch.utils.data import DataLoader, Dataset
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [24]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]
#real_datalist = ["yacht_hydrodynamics"]

syn_datalist = ["syn1"]


missingtypelist = ["quantile","diffuse","logistic"]
#missingtypelist = ["diffuse"]

In [3]:
def load_train_test(dataname,missingtype,rule_name,seed = 1):  
            
    directory_path = f"../datasets/{dataname}"  
    # Opening JSON file
    f = open(f'{directory_path}/split_index_seed-{seed}.json')
    data = json.load(f)

    train_index = data["train_index"]
    test_index = data["test_index"]

    norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
    observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]

    return train_values,train_masks,test_values,test_masks
    

In [38]:
def mean_imp(missingtypelist,datalist_name):


    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        print(missingtype)
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")



        for dataname in datalist:
            path = f"../impute/{missingtype}/{dataname}/Mean"
            if not os.path.exists(path):
                # If the path does not exist, create it
                os.makedirs(path)

            for rule_name in missing_rule:
                train_values,train_masks,test_values,test_masks = load_train_test(dataname,missingtype,rule_name,seed = 1)

                train_values_na = np.where(train_masks == 0, np.nan, train_values)
                test_values_na = np.where(test_masks == 0, np.nan, test_values)
                
                imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
                imp_mean.fit(train_values_na)
                test_imp = imp_mean.transform(test_values_na)
                np.save(f'{path}/{rule_name}_train.npy', test_imp.astype("float32"))
                np.save(f'{path}/{rule_name}_test.npy', test_imp.astype("float32"))


            

In [39]:
mean_imp(missingtypelist,"syn")

quantile
diffuse
logistic
